In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from data import SentenceDataset

In [3]:
en = []
with open('/home/trnmah/final_projectDL/src/MT/data/train-en-vi/train.en', 'r', encoding='utf-8') as file:
	for line in file:
		en.append(line.strip())  # strip() removes trailing newline characters

vi = []
with open('/home/trnmah/final_projectDL/src/MT/data/train-en-vi/train.vi', 'r', encoding='utf-8') as file:
	for line in file:
		vi.append(line.strip())  # strip() removes trailing newline characters
		
en_valid = []
with open('/home/trnmah/final_projectDL/src/MT/data/dev-2012-en-vi/tst2012.en', 'r', encoding='utf-8') as file:
	for line in file:
		en_valid.append(line.strip())  # strip() removes trailing newline characters

vi_valid = []
with open('/home/trnmah/final_projectDL/src/MT/data/dev-2012-en-vi/tst2012.vi', 'r', encoding='utf-8') as file:
	for line in file:
		vi_valid.append(line.strip())  # strip() removes trailing newline characters

train_data_src = en[2269:(2269+4096)]
train_data_trg= vi[2269:(2269+4096)]
valid_data_src = en_valid[269:(269+512)]
valid_data_trg= vi_valid[269:(269+512)]
test_data_src = en_valid[4:(4+256)]
test_data_trg= vi_valid[4:(4+256)]

In [5]:
print(train_data_src[0])

Amanda Bennett : We need a heroic narrative for death


In [6]:
print(train_data_trg[0])

Amanda Bennett : Cần một câu chuyện hào hùng về cái chết


In [ ]:
def translate_en2vi(en_texts: str, tokenizer_en2vi, model_en2vi) -> str:
    input_ids = tokenizer_en2vi(en_texts, padding=True, return_tensors="pt").to(device_en2vi)
    output_ids = model_en2vi.generate(
        **input_ids,
        decoder_start_token_id=tokenizer_en2vi.lang_code_to_id["vi_VN"],
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )
    vi_texts = tokenizer_en2vi.batch_decode(output_ids, skip_special_tokens=True)
    return vi_texts

In [4]:
tokenizer = AutoTokenizer.from_pretrained("vinai/vinai-translate-en2vi-v2")
model = AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi-v2")

inputs = tokenizer("Hello", return_tensors="pt")
labels = tokenizer('Xin chào', return_tensors="pt").input_ids
outputs = model(**inputs, labels=labels)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)


/home/trnmah/mambaforge/envs/practic1/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
inputs

{'input_ids': tensor([[ 8027,     2, 66750]]), 'attention_mask': tensor([[1, 1, 1]])}

In [48]:
outputs.loss

tensor(3.0867, grad_fn=<NllLossBackward0>)

In [10]:
help(AutoModelForSeq2SeqLM.from_pretrained("vinai/vinai-translate-en2vi-v2"))

Help on MBartForConditionalGeneration in module transformers.models.mbart.modeling_mbart object:

class MBartForConditionalGeneration(MBartPreTrainedModel)
 |  MBartForConditionalGeneration(config: transformers.models.mbart.configuration_mbart.MBartConfig)
 |  
 |  The MBART Model with a language modeling head. Can be used for summarization, after fine-tuning the pretrained models.
 |  This model inherits from [`PreTrainedModel`]. Check the superclass documentation for the generic methods the
 |  library implements for all its model (such as downloading or saving, resizing the input embeddings, pruning heads
 |  etc.)
 |  
 |  This model is also a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) subclass.
 |  Use it as a regular PyTorch Module and refer to the PyTorch documentation for all matter related to general usage
 |  and behavior.
 |  
 |  Parameters:
 |      config ([`MBartConfig`]):
 |          Model configuration class with all the parameter

In [47]:
tokenizer.decode(torch.argmax(outputs.logits, dim=2).squeeze())

'Xin chào</s>vi_VN'

In [17]:
tokenizer.decode(inputs['input_ids'][0])

'<s>Hello</s>'

In [ ]:
outputs = model(**inputs, labels=labels)

In [28]:
torch.argmax(outputs.logits, dim=2)

tensor([[    0, 31414,     2,     2,     2,     2]])

In [27]:
tokenizer.decode(torch.argmax(outputs.logits, dim=2))

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

In [32]:
token = tokenizer(["hello", 'hi', 'how are you'], return_tensors="pt", padding='max_length', truncation=True, max_length=5)

In [37]:
token['input_ids'][0].shape

torch.Size([5])

In [41]:
tokenizer.decode(token['input_ids'][2])

'how are you</s>en_XX'

In [4]:
dataset = SentenceDataset(train_data_src, train_data_trg, tokenizer, 128)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
for batch in train_loader:
	# print(batch['src']['input_ids'].shape)
	# print(batch['tgt']['attention_mask'].shape)
	print(batch[0]['input_ids'].shape)
	break

AttributeError: 

In [14]:
dataset[3:5]['src']['input_ids'].shape

torch.Size([2, 128])

In [10]:
tokenizer.tgt_lang = "vi_VN"

In [19]:
inputs = tokenizer("Hello", text_target='Xin chào', return_tensors="pt")

In [20]:
output = model(**inputs)

In [21]:
output.keys()

odict_keys(['loss', 'logits', 'encoder_last_hidden_state'])

In [22]:
output.loss

tensor(0.3004, grad_fn=<NllLossBackward0>)